In [1]:
# Imports
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [2]:
# Setting the sequence length
seq_len = 50

In [3]:
# Loading in Shanties lyrics corpus
shanties = open('../../../data/shanties_all.txt', encoding='utf-8').read()

### Converting Characters to Integers

In [4]:
# Creating a list of all unique characters
chars_list = sorted(list(set(shanties)))

In [5]:
# Creating a dictionary to map each unique character to a number
chars_to_ints = dict((c, i) for i, c in enumerate(chars_list))

In [6]:
# Checking length of corpus and unique characters
len_shanties = len(shanties)
n_chars = len(chars_list)

print(f'Total length of corpus  :  {len_shanties}')
print(f'Total unique characters :  {n_chars}')

Total length of corpus  :  696418
Total unique characters :  27


In [7]:
# Creating a list of patterns for the entire corpus

X_data = []
y_data = []
for i in range(0, len_shanties - seq_len, 1):
    seq_in = shanties[i:i + seq_len]
    seq_out = shanties[i + seq_len]
    X_data.append([chars_to_ints[char] for char in seq_in])
    y_data.append(chars_to_ints[seq_out])

total_patterns = len(X_data)
print(f'Total number of {seq_len} character lenght patters: {total_patterns}')

Total number of 50 character lenght patters: 696368


In [8]:
# Reshaping Data for use in LSTM networks
X = np.reshape(X_data, (total_patterns, seq_len, 1))

# Normalzing X data
X = X / float(n_chars)

# One hot encode to the output variable
y = np_utils.to_categorical(y_data)

### Model Checkpoints

In [9]:
# Creating a checkpoint to find best weights
checkpoint_name = './model-weights/' + str(seq_len) + '-char-sequence/' + str(seq_len) + '-char-seq-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [10]:
'./model-weights/' + str(seq_len) + '-char-sequence/' + str(seq_len) + '-char-seq-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'

'./model-weights/50-char-sequence/50-char-seq-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'

### Building and Running Neural Network

In [11]:
# Defining LSTM model
model = Sequential()

# Adding layers
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

# Compiling model
model.compile(loss='categorical_crossentropy', optimizer='adam')

2021-11-14 21:01:09.672340: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 21:01:09.716864: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 21:01:09.716998: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 21:01:09.717956: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
# Fit model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20


2021-11-14 21:01:19.696880: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8300
2021-11-14 21:01:19.831825: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


5441/5441 [==============================] - ETA: 0s - loss: 2.3377
Epoch 00001: loss improved from inf to 2.33771, saving model to ./model-weights/50-char-sequence/50-char-seq-weights-improvement-01-2.3377.hdf5
5441/5441 [==============================] - 63s 11ms/step - loss: 2.3377
Epoch 2/20
5440/5441 [============================>.] - ETA: 0s - loss: 1.9722
Epoch 00002: loss improved from 2.33771 to 1.97222, saving model to ./model-weights/50-char-sequence/50-char-seq-weights-improvement-02-1.9722.hdf5
5441/5441 [==============================] - 61s 11ms/step - loss: 1.9722
Epoch 3/20
5441/5441 [==============================] - ETA: 0s - loss: 1.8376
Epoch 00003: loss improved from 1.97222 to 1.83760, saving model to ./model-weights/50-char-sequence/50-char-seq-weights-improvement-03-1.8376.hdf5
5441/5441 [==============================] - 60s 11ms/step - loss: 1.8376
Epoch 4/20
5441/5441 [==============================] - ETA: 0s - loss: 1.7559
Epoch 00004: loss improved from 1

In [ ]:
# Saving Model
model.save(f'./models/{seq_len}-char-seq-shanty_writer.h5')